<a href="https://colab.research.google.com/github/MironenkovaEV/-/blob/main/%D1%81%D0%B0%D1%82%D1%83%D1%80%D0%BD_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 13.2 MB/s eta 0:00:00


In [2]:
!wget -nc https://s3-us-west-2.amazonaws.com/ray-tutorials/hackernews.zip
!unzip -o hackernews.zip
!head -n 2 submission-1.json

--2023-11-22 11:50:53--  https://s3-us-west-2.amazonaws.com/ray-tutorials/hackernews.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.132.160, 52.92.225.16, 52.92.138.88, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.132.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56402193 (54M) [application/zip]
Saving to: ‘hackernews.zip’

hackernews.zip      100%[===================>]  53.79M  30.6MB/s    in 1.8s    

2023-11-22 11:50:55 (30.6 MB/s) - ‘hackernews.zip’ saved [56402193/56402193]

Archive:  hackernews.zip
  inflating: submission-1.json       
  inflating: submission-2.json       
  inflating: submission-3.json       
  inflating: submission-4.json       
{"body": {"descendants": 0, "url": "http://markpincus.blogspot.com/2005/03/peopleweb-i-believe-we-are-close-to.html", "text": "", "title": "The PeopleWeb | Mark Pincus Blog (March 2005)", "by": "sayemm", "score": 3, "time": 1286515576, "

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import numpy as np
import pandas as pd
import ray
import time

In [4]:
ray.init(num_cpus=8, ignore_reinit_error=True)
#ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

2023-11-22 11:51:08,545	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.8.0


In [5]:
def parse_hn_submissions(path):
    with open(path, "r") as f:
        records = []
        for line in f.readlines():
            body = json.loads(line)["body"]
            records.append({"data": body["title"], "score": body["score"]})
        return pd.DataFrame(records)

In [6]:
start_time = time.time()

files = ["submission-" + str(i) + ".json" for i in range(1, 5)]
records = [parse_hn_submissions(file) for file in files]
df = pd.concat(records)

end_time = time.time()
duration = end_time - start_time
print("Took {} seconds to parse the hackernews submissions".format(duration))

df.head()

Took 6.100790977478027 seconds to parse the hackernews submissions


,data,score
0,The PeopleWeb | Mark Pincus Blog (March 2005),3
1,Computer science and programming are two separ...,1
2,Don't Go It Alone: Create an Advisory Board,1
3,Wikileaks Secret Dreams,1
4,MakeMyTrip.com: Is eCommerce in India Finall...,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   data    400000 non-null  object
 1   score   400000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.2+ MB


In [8]:
df.describe()

,score
count,400000.000000
mean,9.294667
std,37.853269
min,0.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,2376.000000


In [9]:
df["score"].median()

1.0

In [10]:
df["target"] = df["score"] > 1.0

In [11]:
df.head()

,data,score,target
0,The PeopleWeb | Mark Pincus Blog (March 2005),3,True
1,Computer science and programming are two separ...,1,False
2,Don't Go It Alone: Create an Advisory Board,1,False
3,Wikileaks Secret Dreams,1,False
4,MakeMyTrip.com: Is eCommerce in India Finall...,1,False


In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [ ]:
train

,data,score,target
59049,Amusing fake job post by Innocent Drinks,2,True
95896,Is PHP.net down? 502 Error?,1,False
33241,Can You Survive a Benevolent Dictatorship? (Do...,4,True
65370,Lead Acid Batteries: How Cheap Beat Cool at Go...,4,True
54308,State of the Internet,2,True
...,...,...,...
88252,French Courts Fine eBay For Buying Typo Keywords,1,False
76087,"Results Unproven, Robotic Surgery Wins Converts",1,False
13471,The perceived value should be as high as possi...,1,False
55315,HP tablet has integrated social networking bui...,1,False


In [ ]:
test

,data,score,target
4383,Atolo – Free Responsive Single Page Template,1,False
63497,Richard Branson accepting micropitches via Twi...,36,True
75291,Boxee's thoughts about the Apple TV,24,True
4470,اخبار الحوادث,1,False
10862,KitchenAid Mixer Artisan Series,1,False
...,...,...,...
83003,"Onefinestay, The Upmarket Airbnb, Expands Its ...",1,False
92692,List of the biggest private companies in the US,2,True
81817,Deviance in the Dark,1,False
94352,Modular Phones Invented Before Google's Projec...,1,False


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("sgd", SGDClassifier(loss="hinge", penalty="l2",
                          alpha=0.001,
                          max_iter=10000, tol=1e-3,
                          warm_start=True))])
result = pipeline.fit(train.data, train.target)

predicted = result.predict(train.data)
print("Accuracy on the training set is {}".format(np.mean(predicted == train.target)))

Accuracy on the training set is 0.585071875


In [14]:
predicted = pipeline.predict(test.data)
print("Accuracy on the test set is {}".format(np.mean(predicted == test.target)))

Accuracy on the test set is 0.5775


In [15]:
pipeline.predict(["Iconic consoles of the IBM System/360 mainframes, 55 years old today",
                  "Are Banned Drugs in Your Meat?"])

array([ True, False])

In [16]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [17]:
import os
import pickle
from ray import tune

In [18]:
train_id = ray.put(train)
test_id = ray.put(test)

def train_func(config, reporter):
    pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("sgd", SGDClassifier(loss="hinge", penalty="l2",
                          alpha=config["alpha"],
                          #alpha=0.001,
                          max_iter=1000, tol=1e-3,
                          warm_start=True))])
    # TODO: Put in the training pipeline here
    train = ray.get(train_id)
    test = ray.get(test_id)
    for i in range(1000):
        # Perform one epoch of SGD
        X = pipeline.named_steps["vect"].fit_transform(train.data)
        pipeline.named_steps["clf"].partial_fit(X, train.target, classes=[0, 1])
        with open("model.pkl", "wb") as f:
            pickle.dump(pipeline, f)
        reporter(mean_accuracy=np.mean(pipeline.predict(test.data) == test.target))  # report metrics

In [19]:
from ray import train, tune

def train_fn(config):
    train.report({"alpha": tune.grid_search([1e-3, 1e-4, 1e-5, 1e-6])})

tuner = tune.Tuner(train_fn)
all_trials = tuner.fit()

2023-11-22 11:54:03,935	INFO tune.py:595 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-11-22 11:54:03,951	INFO tensorboardx.py:178 -- pip install "ray[tune]" to see TensorBoard files.
2023-11-22 11:54:03,954	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


+-----------------------------------------------------------------+
| Configuration for experiment     train_fn_2023-11-22_11-54-03   |
+-----------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator          |
| Scheduler                        FIFOScheduler                  |
| Number of trials                 1                              |
+-----------------------------------------------------------------+

View detailed results here: /root/ray_results/train_fn_2023-11-22_11-54-03

Trial status: 1 PENDING
Current time: 2023-11-22 11:54:04. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+---------------------------------+
| Trial name             status   |
+---------------------------------+
| train_fn_d5428_00000   PENDING  |
+---------------------------------+

Trial train_fn_d5428_00000 started without custom configuration.

Trial train_fn_d5428_00000 completed after 1 iterations at 2023-11-22 11:54

In [20]:
all_trials.get_best_result()

Result(
  metrics={'alpha': {'grid_search': [0.001, 0.0001, 1e-05, 1e-06]}},
  path='/root/ray_results/train_fn_2023-11-22_11-54-03/train_fn_d5428_00000_0_2023-11-22_11-54-03',
  filesystem='local',
  checkpoint=None
)

In [21]:
#tuner.get_results()

In [22]:
'''
best_result_path = os.path.join(all_trials.get_best_logdir("mean_accuracy"), "model.pkl")
with open(best_result_path, "rb") as f:
    pipeline = pickle.load(f)
print("Best result was {}".format(np.mean(pipeline.predict(test.data) == test.target)))
print("Best result path is {}".format(best_result_path))
'''

'\nbest_result_path = os.path.join(all_trials.get_best_logdir("mean_accuracy"), "model.pkl")\nwith open(best_result_path, "rb") as f:\n    pipeline = pickle.load(f)\nprint("Best result was {}".format(np.mean(pipeline.predict(test.data) == test.target)))\nprint("Best result path is {}".format(best_result_path))\n'